# What affects methane lifetime?

- methane
- VOCs
- NOx
- Ozone
- HCs
- N2O
- climate

Here I suggest an override of the alpha scaling factor for methane that is calculated from multiple species.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats

from tqdm import tqdm
from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, FAIR
from fair21.defaults import species_config_from_default

SSP3-7.0 climate

In [ ]:
df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(751)
gmst[100:271] = df_temp['gmst'].values
gmst[271:351] = np.linspace(gmst[270], 4, 80)
gmst[351:] = 4

In [ ]:
samples = 500

Here we need to include all methane lifetime precursors.
For now I will exclude HC, effect is small. 

Do I need to include ozone?

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
conc_species = ['CH4', 'N2O']
conc_in = {}
for species in conc_species:
    conc_in[species] = df_conc.loc[
        (df_conc['Scenario']=='ssp370') & (df_conc['Variable'].str.endswith(species)) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
emis_species = ['CO', 'VOC', 'NOx']
emis_in = {}
for species in emis_species:
    emis_in[species] = df_emis.loc[
        (df_emis['Scenario']=='ssp370') & (df_emis['Variable'].str.endswith(species)) & 
        (df_emis['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
# species_ids = {
#     'ch4': SpeciesID('CH4', Category.CH4),
#     'co2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
#     'n2o': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
#     'co': SpeciesID('CO', Category.SLCF_OZONE_PRECURSOR),
#     'voc': SpeciesID('VOC', Category.SLCF_OZONE_PRECURSOR),
#     'nox': SpeciesID('NOx', Category.SLCF_OZONE_PRECURSOR),
# }

In [ ]:
emis_in

In [ ]:
def alpha_scaling(
    emis,
    conc,
    emis_pi,
    conc_pi,
    temperature,
    beta_ch4,
    beta_n2o,
    beta_co,
    beta_voc,
    beta_nox,
    beta_temperature
):
    (conc['CH4']-conc_pi['CH4']) * beta_ch4